From https://scalexi.medium.com/implementing-a-retrieval-augmented-generation-rag-system-with-openais-api-using-langchain-ab39b60b4d9f

In [2]:
%conda install faiss-gpu hnswlib
%pip install langchain openai tiktoken langchain_experimental "langchain[docarray]" python-dotenv

Retrieving notices: ...working... done
Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
import os
import pickle
from dotenv import load_dotenv
# shouldn't be needed, apparently jupyter does this on his own...
load_dotenv()
if(os.environ["OPENAI_API_KEY"]) : print( "API key loaded")

API key loaded


In [4]:
def getEmbeddings():
    cache = './data/cache/articleembeddings_oaitut.pkl'
    try:
        with open(cache,"rb") as inp:
            return pickle.load(inp)
    except IOError:
        loader = TextLoader("./data/samplearticle.txt",encoding="utf-8")
        data=loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        data = text_splitter.split_documents(data)
        embeddings = OpenAIEmbeddings()
        vectorstore = FAISS.from_documents(data, embedding=embeddings)
        with open(cache,"wb") as out:
            pickle.dump(vectorstore,out,pickle.DEFAULT_PROTOCOL)
        return vectorstore
vectorstore = getEmbeddings()

Created a chunk of size 1064, which is longer than the specified 1000
Created a chunk of size 1213, which is longer than the specified 1000
Created a chunk of size 1059, which is longer than the specified 1000
/workspaces/ragtest/.conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [ ]:
query = "What configuration do I need?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer